In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum, format_number
spark = SparkSession.builder.appName("Ques 7").getOrCreate()

DF=spark.read.csv("house_apartment_cleaned_data.csv")
# DF.show()
columns = ["locality", "types_of_property", "subtype_of_property", "price", "no_of_rooms", "open_fire", "terrace", 
           "garden","swimming_pool", "state_building", "construction_year"]
# print(columns)

mydf=DF.toDF(*columns)
# mydf.show()

In [3]:
houseWithTerrace = mydf.filter((col("types_of_property") == "house") & (col("terrace") == 1))
# houseWithTerrace.show()
totalWithPrice = houseWithTerrace.groupBy("subtype_of_property").agg(sum("price").alias("Total_price_with_Terrace"))
formatted_with_price = totalWithPrice.withColumn("Total_price_with_Terrace", format_number("Total_price_with_Terrace", 0))
# formatted_with_price.show()

houseWithoutTerrace = mydf.filter((col("types_of_property") == "house") & (col("terrace") == 0))
# houseWithoutTerrace.show()
totalWithoutPrice = houseWithoutTerrace.groupBy("subtype_of_property").agg(sum("price").alias("Total_price_without_Terrace"))
formatted_without_price = totalWithoutPrice.withColumn("Total_price_without_Terrace", format_number("Total_price_without_Terrace", 0))
# formatted_without_price.show()


merge = (formatted_with_price
             .join(formatted_without_price, formatted_with_price["subtype_of_property"] == formatted_without_price["subtype_of_property"])
             )
new_df = merge.select(formatted_with_price["subtype_of_property"], "Total_price_with_Terrace", "Total_price_without_Terrace")
new_df.show()

+--------------------+------------------------+---------------------------+
| subtype_of_property|Total_price_with_Terrace|Total_price_without_Terrace|
+--------------------+------------------------+---------------------------+
|         manor house|              16,566,000|                 28,025,500|
|               villa|             909,476,874|                477,007,539|
|           farmhouse|              24,985,500|                 37,995,009|
|              castle|               3,749,999|                  2,539,000|
|  mixed use building|             173,179,254|                185,599,482|
|     apartment block|             185,668,603|                240,367,941|
|              chalet|              10,740,200|                  6,789,856|
|            bungalow|              50,304,694|                 27,411,250|
|      other property|               7,719,786|                 15,315,500|
|             mansion|             127,880,590|                 92,403,895|
|     countr